Todos os imports usados no notebook

In [82]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

1. Preparação dos Dados (ETL)
1.1. Conectar ao MongoDB:

Utilize pymongo para se conectar ao MongoDB e acessar as coleções.

In [83]:
client = MongoClient('mongodb://localhost:27017/')
db = client['tcc-data-analysis']

1.2. Extrair Dados:

Extraia os dados das coleções necessárias (commits, react_prs, react_bugs, commit_diffs).

In [84]:
commits = db['commits'].find()
prs = db['react_prs'].find()
bugs = db['react_bugs'].find()
diffs = db['commit_diffs'].find()
commits_prs = db['commits_prs'].find()

1.3. Limpeza e Transformação:

Normalize e limpe os dados. Crie um DataFrame para cada coleção com as informações relevantes.

In [85]:
_commits = list(commits)
_diffs = list(diffs)
commits_df = pd.json_normalize(_commits)
diffs_df = pd.json_normalize(_diffs)
commits_df.rename(columns={'sha': 'commit_sha'}, inplace=True)
diffs_df.rename(columns={'commit': 'commit_sha'}, inplace=True)


In [86]:
combined_df = pd.merge(commits_df, diffs_df, on='commit_sha', how='inner')

In [87]:
combined_df = combined_df.dropna(subset=['diff'])

Index(['_id_x', 'commit_sha', 'node_id', 'url', 'html_url', 'comments_url',
       'parents', 'commit.author.name', 'commit.author.email',
       'commit.author.date', 'commit.committer.name', 'commit.committer.email',
       'commit.committer.date', 'commit.message', 'commit.tree.sha',
       'commit.tree.url', 'commit.url', 'commit.comment_count',
       'commit.verification.verified', 'commit.verification.reason',
       'commit.verification.signature', 'commit.verification.payload',
       'author.login', 'author.id', 'author.node_id', 'author.avatar_url',
       'author.gravatar_id', 'author.url', 'author.html_url',
       'author.followers_url', 'author.following_url', 'author.gists_url',
       'author.starred_url', 'author.subscriptions_url',
       'author.organizations_url', 'author.repos_url', 'author.events_url',
       'author.received_events_url', 'author.type', 'author.site_admin',
       'committer.login', 'committer.id', 'committer.node_id',
       'committer.avatar_ur

1.4. Integração de Dados:

Combine dados de diferentes coleções conforme necessário. Por exemplo, associe commits com PRs e bugs usando o hash do commit.

In [88]:
pd_commit = pd.DataFrame((combined_df))
pd_issues = pd.DataFrame((bugs))
pd_prs = pd.DataFrame((prs))
pd_commits_prs = pd.DataFrame((commits_prs))

In [89]:
print('pd_commit', pd_commit.columns)
print('pd_issues', pd_issues.columns)
print('pd_prs', pd_prs.columns)
print('pd_commits_prs', pd_commits_prs.columns)


pd_commit Index(['_id_x', 'commit_sha', 'node_id', 'url', 'html_url', 'comments_url',
       'parents', 'commit.author.name', 'commit.author.email',
       'commit.author.date', 'commit.committer.name', 'commit.committer.email',
       'commit.committer.date', 'commit.message', 'commit.tree.sha',
       'commit.tree.url', 'commit.url', 'commit.comment_count',
       'commit.verification.verified', 'commit.verification.reason',
       'commit.verification.signature', 'commit.verification.payload',
       'author.login', 'author.id', 'author.node_id', 'author.avatar_url',
       'author.gravatar_id', 'author.url', 'author.html_url',
       'author.followers_url', 'author.following_url', 'author.gists_url',
       'author.starred_url', 'author.subscriptions_url',
       'author.organizations_url', 'author.repos_url', 'author.events_url',
       'author.received_events_url', 'author.type', 'author.site_admin',
       'committer.login', 'committer.id', 'committer.node_id',
       'committer

2. Análise Exploratória de Dados (EDA)
2.1. Visualização:

Utilize matplotlib e seaborn para visualizar distribuições e padrões.

In [90]:
#sns.histplot(combined_df['date'])
#plt.show()

2.2. Análise Estatística:

Analise a correlação entre features e a presença de bugs.
3. Engenharia de Recursos
3.1. Extração de Features:

Extraia features úteis dos dados, como a frequência de commits de um autor, a complexidade dos diffs, e a presença de palavras-chave nos commits.

In [91]:
combined_df['message_length'] = combined_df['message'].apply(len)

KeyError: 'message'

3.2. Criação de Labels:

Defina o rótulo (label) que indica se um commit causou um bug ou não. Combine informações de react_bugs para determinar isso.
3.3. Preparação dos Dados:

Divida os dados em conjuntos de treinamento e teste. Normalize os dados se necessário.

In [ ]:
X = combined_df[['message_length']]  # Exemplo de feature
y = combined_df['bug']  # Supondo que você tenha um rótulo 'bug'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


KeyError: 'bug'

4. Seleção e Treinamento do Modelo
4.1. Escolher Modelos:

Comece com modelos simples como Logistic Regression e Decision Trees. Avalie a performance usando métricas apropriadas (ex.: precisão, recall).

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


4.2. Avaliação e Ajuste:

Experimente diferentes modelos e ajuste hiperparâmetros para melhorar a performance.
5. Validação e Implementação
5.1. Validação Cruzada:

Use validação cruzada para garantir que o modelo generaliza bem para novos dados.

In [ ]:

scores = cross_val_score(model, X, y, cv=5)
print(scores.mean())


5.2. Implementação:

Implemente o modelo em um ambiente de produção e crie uma interface para interagir com o modelo.
6. Monitoramento e Manutenção
6.1. Monitoramento:

Monitore o desempenho do modelo e ajuste conforme necessário.
6.2. Atualização:

Atualize o modelo com novos dados regularmente para manter a precisão.